# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [3]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [4]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [5]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [6]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

In [7]:
sigma[1,: ,:]

# cov matrix of label 1 wine

array([[ 2.33252785e-01, -1.35961601e-02, -3.93531639e-03,
        -3.13598161e-01,  1.05226609e+00,  6.06773391e-02,
         7.52687399e-02,  4.65613845e-03,  6.21497566e-02,
         2.21752244e-01,  1.14922120e-02, -1.16165495e-03,
         4.04223580e+01],
       [-1.35961601e-02,  4.31329475e-01, -9.77187669e-03,
         2.38159546e-01, -2.44040022e-01, -1.37782044e-02,
        -4.24053002e-02, -1.89085992e-03, -5.46760411e-02,
        -2.14098215e-01, -3.71030827e-02,  1.33378042e-02,
        -4.66765279e+01],
       [-3.93531639e-03, -9.77187669e-03,  3.67746890e-02,
         2.35263386e-01,  5.65473229e-01,  3.68015143e-03,
        -1.40778799e-03,  4.47014602e-03, -1.05729584e-02,
         1.23742564e-04,  5.77928610e-03, -5.94402380e-03,
         6.13422390e+00],
       [-3.13598161e-01,  2.38159546e-01,  2.35263386e-01,
         6.04011898e+00,  5.56208761e+00, -1.75295295e-01,
        -3.16022715e-01,  1.40292050e-02, -2.37452136e-01,
        -5.97414278e-01, -1.85613845e

## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [8]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    mu_sub = mu[:,features]
    covar_sub = sigma[:,features,:][:,:,features]
    random_vars = {}
    for label in range (1,4):
        random_vars[label] = multivariate_normal(mean = mu_sub[label,:],cov = covar_sub[label,:,:])
    z = np.zeros(len(ty))
    for i in range (0,len(ty)):
        scores = []
        for label in range (1,4):
            scores.append(np.log(pi[label]) + random_vars[label].logpdf(tx[i,features]))
        z[i] = np.argmax(scores)+1
    err_position = np.not_equal(z,ty)
    err_numb = np.sum(err_position)
    return err_numb,z

In [9]:
sigma[1,[2,4,6],[2,4,6]]

array([3.67746890e-02, 1.18461871e+02, 1.52409410e-01])

In [10]:
test_model(mu, sigma, pi, [2,4,6], testx, testy)


(7, array([1., 2., 3., 3., 1., 1., 3., 1., 3., 2., 1., 1., 2., 1., 2., 2., 3.,
        1., 3., 2., 3., 2., 1., 2., 1., 2., 3., 3., 2., 2., 2., 2., 3., 3.,
        1., 1., 3., 2., 3., 2., 1., 1., 1., 2., 2., 2., 1., 3.]))

In [11]:
testy

array([1., 2., 3., 3., 1., 1., 3., 1., 3., 2., 1., 1., 2., 1., 3., 2., 3.,
       1., 3., 3., 3., 1., 2., 2., 2., 2., 3., 3., 2., 2., 1., 2., 3., 3.,
       1., 1., 3., 2., 2., 2., 1., 1., 1., 2., 2., 2., 1., 3.])

In [12]:
mu,covar,pi = fit_generative_model(trainx,trainy)
mu_sub=mu[:,[2,4,6]]
mu_sub
mu_sub[1]

array([  2.42790698, 105.8372093 ,   2.99627907])

In [13]:
pi

array([0.        , 0.33076923, 0.41538462, 0.25384615])

In [14]:
covar_sub = covar[:,[2,4,6],:]
covar_sub = covar_sub [:,:,[2,4,6]]
covar_sub

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.67746890e-02,  5.65473229e-01, -1.40778799e-03],
        [ 5.65473229e-01,  1.18461871e+02,  6.48464035e-01],
        [-1.40778799e-03,  6.48464035e-01,  1.52409410e-01]],

       [[ 1.12302332e-01,  9.88395062e-01,  9.11620713e-02],
        [ 9.88395062e-01,  3.28509259e+02,  1.97993827e-01],
        [ 9.11620713e-02,  1.97993827e-01,  5.68697291e-01]],

       [[ 2.67597796e-02,  4.35426997e-01,  2.11539945e-02],
        [ 4.35426997e-01,  1.25423324e+02,  1.47190083e+00],
        [ 2.11539945e-02,  1.47190083e+00,  7.37531680e-02]]])

In [15]:
indices = (trainy == 3)
x=trainx[indices,:]
np.mean(trainx[indices,:][:,[2,4,6]], axis=0)


array([ 2.40090909, 99.03030303,  0.75727273])

In [16]:
indices = (trainy == 3)
trainx[indices,:].shape
#np.mean(trainx[indices,[2,4,6]], axis=0)

(33L, 13L)

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [17]:
test_model(mu, sigma, pi, [2], testx, testy)

(29, array([1., 2., 1., 2., 1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 2., 2., 1., 1., 2., 1., 1., 2., 2., 2., 2., 1., 2.,
        1., 1., 1., 1., 2., 1., 2., 1., 2., 2., 2., 2., 2., 1.]))

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [18]:
test_model(mu, sigma, pi, [0,2], testx, testy)

(12, array([1., 2., 3., 2., 1., 2., 3., 1., 3., 2., 1., 1., 2., 1., 3., 2., 2.,
        1., 1., 1., 3., 1., 2., 2., 3., 2., 3., 1., 1., 2., 2., 2., 3., 1.,
        1., 1., 3., 2., 2., 2., 1., 3., 1., 2., 2., 2., 1., 1.]))

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [19]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

(3, array([1., 2., 3., 3., 1., 2., 3., 1., 3., 2., 1., 1., 2., 1., 2., 2., 3.,
        1., 3., 3., 3., 1., 2., 2., 2., 2., 3., 3., 2., 2., 2., 2., 3., 3.,
        1., 1., 3., 2., 2., 2., 1., 1., 1., 2., 2., 2., 1., 3.]))

Exercise 4. How many errors when using all 13 features?

In [20]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

(2, array([1., 2., 3., 3., 1., 1., 3., 1., 3., 2., 1., 1., 2., 1., 3., 2., 3.,
        1., 3., 3., 3., 1., 2., 2., 2., 2., 3., 3., 2., 2., 2., 1., 3., 3.,
        1., 1., 3., 2., 2., 2., 1., 1., 1., 2., 2., 2., 1., 3.]))

Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?